<a href="https://colab.research.google.com/github/norojordan/big-data-challenge/blob/main/amazon_watches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-23 23:45:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.75MB/s    in 0.5s    

2022-05-23 23:45:35 (1.75 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [17]:
from pyspark import SparkFiles
# Load in Amazon watch reviews from S3 into a DataFrame
url = "https://s3.amazonaws.com/nj-hw-bucket/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

watch_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t",inferSchema=True, timestampFormat="yyyy-mm-dd")
watch_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...|2015-01-31 00:08:00|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...| 

In [18]:
# Count the number of reviews 
print(watch_df.count())

960872


## Drop duplicates and incomplete rows

In [25]:
watch_df = watch_df.dropna()
watch_df = watch_df.drop_duplicates()
print(watch_df.count())

960679


## Examine the schema

In [26]:
watch_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create a new DataFrame for review info

In [35]:
review_id_info = watch_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_info.show(5)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1004K426LTDKN|   15688375|B002XUL6YY|     852892099|2014-01-22 00:05:00|
|R1006QGKXQ3Q96|   21739087|B005JVP0LE|     559701518|2014-01-16 00:06:00|
|R1017P0ZGRXSR0|    2501262|B0009P66YK|       6220965|2015-01-26 00:02:00|
|R103HXXDQD9W4Y|   16842563|B008MVVA2Q|     849986272|2014-01-30 00:12:00|
|R10472PWW2S2AW|   52130705|B001D9FGAG|     940154187|2009-01-27 00:01:00|
+--------------+-----------+----------+--------------+-------------------+
only showing top 5 rows



## Create a new DataFrame for products info

In [ ]:
products_info=watch_df.select(["product_id", "product_title"]).drop_duplicates()
products_info.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00C89MF4A|Weide Mens Red Th...|
|B004FD38JS|TAG Heuer Men's W...|
|B00ATUFSY4|Invicta Men's 140...|
|B009BEO81I|        Fossil Riley|
|B008B39LSA|XOXO Women's XO55...|
+----------+--------------------+
only showing top 5 rows



## Create a new DataFrame for customers info

In [ ]:
# Total customers count 
customer_count = watch_df.select(["customer_id"]).count()
customer_count

960679

In [ ]:
customer_watch = watch_df.groupBy('customer_id').count()
customer_watch.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   12937860|    1|
|   14327548|    4|
|   22158149|    5|
|   12828562|    2|
|    4089580|    1|
+-----------+-----+
only showing top 5 rows



In [ ]:
# Change column name to match schema
customer_watch = customer_watch.withColumnRenamed("count", "customer_count") 
customer_watch.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12937860|             1|
|   14327548|             4|
|   22158149|             5|
|   12828562|             2|
|    4089580|             1|
|   20097895|             1|
|    2422277|             4|
|   35523976|             2|
|   14051742|             1|
|   14404180|             5|
|   12883856|             3|
|    9942583|             2|
|   38893812|             1|
|   22754948|             1|
|   51265055|             5|
|   49225180|             1|
|   16215726|             1|
|   39948072|            20|
|   14847189|             3|
|   17207294|             2|
+-----------+--------------+
only showing top 20 rows



## Create a new DataFrame for vine info

In [ ]:
vine_info = watch_df.select(["review_id", "star_rating", "helpful_votes", "total_votes","vine"])
vine_info.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006OPQYUMAHS|          1|            4|          5|   N|
|R1010XUX2AIF10|          5|            0|          0|   N|
|R102DONPK6YEGN|          5|            0|          0|   N|
|R102N48HXFM9WY|          5|            0|          0|   N|
|R1060WX8UX26JV|          2|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



## Write DataFrame to RDS

In [11]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://mypostgreshwdb.cf9s0qdbvngm.us-east-1.rds.amazonaws.com/my_data_HW_db"
config = {"user":"xxx",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [36]:
# Write DataFrame to pgAdmin table

review_id_info.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

products_info.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

customer_watch.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

vine_info.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)